In [1]:
today = "2024-01-30"
month = "12 dec 2023"  # note: used to retrieve sheet from coco trainer data

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import os


def load_pending_df(path_: str) -> pd.DataFrame:
    """Load pending result DF."""

    df = pd.read_excel(path_, skiprows=1, engine="xlrd")
    return df


def load_all_pending_dfs(path_: str) -> list[pd.DataFrame]:
    """Load all pending result DFs, return them as a list of DFs."""

    offline_classroom = load_pending_df(Path(path_, "Pending Results.xls"))
    offline_other = load_pending_df(Path(path_, "Pending Results (1).xls"))
    online_classroom = load_pending_df(Path(path_, "Pending Results (2).xls"))
    online_other = load_pending_df(Path(path_, "Pending Results (3).xls"))
    return [offline_classroom, offline_other, online_classroom, online_other]


def load_trainer_df(month: str) -> pd.DataFrame:
    """Load trainer DF which details the area of ET."""

    path = "/home/anj/Documents/wse-local/2. Experience/dependencies/coco_trainer_data.xlsx"
    df_trainer = pd.read_excel(path, sheet_name=month)

    return df_trainer


def clean_trainer_name(df: pd.DataFrame, teacher_col: str) -> pd.Series:
    """Clean teacher's name."""

    teachers = (
        df[teacher_col]
        .str.title()
        .str.replace("\(.+\)", "", regex=True)
        .str.replace("\s+", " ", regex=True)
        .str.strip()
        .replace(
            {
                "Azhar Rahul": "Azhar Rahul Finaya",
                "Handayani Risma": "Handayani Khaerunisyah Risma",
                "Kartikasari Prettya": "Kartikasari Prettya Nur",
                "Ramadhan Ira Ragil": "Ramadhani Ira",
                "S Allan": "Santiago Allan",
                "Gandhama Jesita": "Ghandama Jesita",
                "Istiqomah Diah": "Toluhula Diah Istiqomah",
                "Putri Tiara": "Setiawan Tiara Putri",
                "Hamsah Ratnasari Handayani": "Hamsah Handayani Ratnasari",
            }
        )
    )
    return teachers


def clean_pending_df(dfs: list, date_exported: str, month: str) -> pd.DataFrame:
    """Clean pending dfs to obtain list of pending results per session."""

    df_clean = (
        # concat dfs that is obtained from load_all_pending_dfs
        pd.concat(dfs)
        # drop unused columns
        .drop(
            columns=["Level / Unit", "First Name", "Last Name", "Code", "Service Type"]
        )
        # drop duplicates based on this subset to get per session
        .drop_duplicates(
            subset=["Teacher", "Class Type", "Date", "Start Time"], keep="first"
        )
        # rename columns
        .rename(columns=lambda c: c.lower().replace("_", " "))
        # drop na rows and cols
        .dropna(subset=["teacher"])
        .dropna(how="all", axis=0)
        .dropna(how="all", axis=1)
        .assign(
            teacher=lambda df_: clean_trainer_name(
                df_, "teacher"
            ),  # clean teacher name
            date=lambda df_: pd.to_datetime(df_["date"]),  # get the clean date
            date_exported=pd.to_datetime(date_exported),  # data exported date
        )
        # merge with et data to get area and position
        .merge(
            right=load_trainer_df(month),
            left_on="teacher",
            right_on="teacher",
            how="left",
        )
        # drop unused cols
        .drop(
            columns=[
                "teacher_working_days",
                "teacher_note_1",
                "teacher_note_2",
                "center name",
            ]
        )
        # sort columns
        .loc[
            :,
            [
                "teacher",
                "date",
                "start time",
                "class type",
                "teacher_position",
                "teacher_center",
                "teacher_area",
                "date_exported",
            ],
        ]
        # sort rows
        .sort_values(["teacher_area", "teacher_center", "teacher", "date"])
        .reset_index(drop=True)
    )

    return df_clean


def count_pending_result(df: pd.DataFrame, today: str) -> pd.DataFrame:
    """Get summary of pending result in the last 365 days,
    grouped by area and teacher, pivoted per month.
    """

    return (
        df
        # filter only pending result for the past 365 days
        .loc[lambda df_: (pd.to_datetime(today) - df_["date"]).dt.days <= 365]
        # group
        .groupby(["teacher_area", "teacher", pd.Grouper(key="date", freq="1M")])
        .agg(num_session_with_pending_res=("teacher", "count"))
        .reset_index()
        # pivot
        .pivot(index=["teacher_area", "teacher"], columns="date")
        # fill na with 0
        .fillna(0)
        # note: do not display trainer if the last 3 months pending results is 0
        .loc[lambda df_: np.sum(df_.iloc[:, -3:], axis=1) > 0]
        # clean col names and all
        .droplevel(0, axis=1)
        .rename(columns=lambda c: c.strftime("%b %Y"))
        .rename_axis(["Teacher Area", "Teacher"])
        .rename_axis([""], axis=1)
    )

In [3]:
# load all pending dfs in a folder
dfs = load_all_pending_dfs(Path("data", today))
# clean data
df_clean = clean_pending_df(dfs, today, month)
# count pending result per month / create summary
df_pending = count_pending_result(df_clean, today)

WARNING *** file size (287331) not 512 + multiple of sector size (512)
WARNING *** file size (1303139) not 512 + multiple of sector size (512)
WARNING *** file size (6819) not 512 + multiple of sector size (512)
WARNING *** file size (402019) not 512 + multiple of sector size (512)


In [4]:
def test_all_teacher_exist_in_coco_trainer_data():
    unmapped = df_clean.loc[df_clean["teacher_area"].isna(), "teacher"].unique()
    assert (
        unmapped.shape[0] == 0
    ), f"some teacher are not listed in coco_trainer_data: {unmapped}"


test_all_teacher_exist_in_coco_trainer_data()

In [5]:
# create data per area
df_clean = df_clean.rename(columns=lambda c: c.replace("_", " ").title())
df_jkt1 = df_clean.loc[df_clean["Teacher Area"] == "JKT 1"]
df_jkt2 = df_clean.loc[df_clean["Teacher Area"] == "JKT 2"]
df_jkt3 = df_clean.loc[df_clean["Teacher Area"] == "JKT 3"]
df_sby = df_clean.loc[df_clean["Teacher Area"] == "SBY"]
df_bdg = df_clean.loc[df_clean["Teacher Area"] == "BDG"]
df_onl = df_clean.loc[
    df_clean["Teacher Area"].isin(["Online", "Shared Account", "Ooolab"])
]
df_oth = df_clean.loc[df_clean["Teacher Area"] == "Other"]

# assert that no rows are missed
assert len(df_clean) == sum(
    [len(df) for df in [df_jkt1, df_jkt2, df_jkt3, df_sby, df_bdg, df_onl, df_oth]]
)

In [6]:
# save df
# write each dataframe to a different worksheet.

filename = "output.xlsx"
filepath = os.path.join("data", today, filename)
writer = pd.ExcelWriter(filepath, engine="xlsxwriter")

df_pending.to_excel(writer, sheet_name="Summary", index=True)
df_clean.to_excel(writer, sheet_name="All Area", index=False)
df_jkt1.to_excel(writer, sheet_name="JKT 1", index=False)
df_jkt2.to_excel(writer, sheet_name="JKT 2", index=False)
df_jkt3.to_excel(writer, sheet_name="JKT 3", index=False)
df_sby.to_excel(writer, sheet_name="SBY", index=False)
df_bdg.to_excel(writer, sheet_name="BDG", index=False)
df_onl.to_excel(writer, sheet_name="Online", index=False)
df_oth.to_excel(writer, sheet_name="Other", index=False)

writer.close()